In [130]:
import random
from datetime import datetime, timedelta
from app.db import get_db_connection

In [131]:
conn = await get_db_connection()

Conexão com o banco de dados estabelecida!


In [123]:
query_base = """
SELECT DISTINCT
    unidade_executante,
    profissional,
    especialidade,
    solicitante,
    id_usuario,
    nome_usuario,
    template_id,
    tipo_envio,
    exame
FROM public.cross_agendamentos
WHERE solicitante IS NOT NULL
"""

base_rows = await conn.fetch(query_base)

print(f"Solicitantes encontrados: {len(base_rows)}")

Solicitantes encontrados: 106


In [124]:
def random_date(start, end):
    delta = end - start
    return start + timedelta(days=random.randint(0, delta.days))


def random_time():
    return (datetime.min + timedelta(minutes=random.randint(420, 1020))).time()

In [125]:
TOTAL_REGISTROS = 500

data_inicio = datetime(2025, 1, 1)
data_fim = datetime(2026, 2, 1)

In [126]:
registros = []

for i in range(TOTAL_REGISTROS):
    row = random.choice(base_rows)

    registros.append((
        row["unidade_executante"],
        row["profissional"],
        random_date(data_inicio, data_fim).date(),
        row["especialidade"],
        random_time(),

        random.randint(100000, 999999),

        f"PACIENTE TESTE {i+1}",
        f"119{random.randint(10000000,99999999)}",

        random.choice(["CONFIRMO", "NÃOㅤCONFIRMO", "NÃOㅤCONHEÇO"]),

        row["solicitante"],
        row["id_usuario"],
        row["nome_usuario"],
        row["template_id"],
        row["tipo_envio"],
        row["exame"]
    ))

In [127]:
insert_sql = """
INSERT INTO public.cross_agendamentos (
    unidade_executante,
    profissional,
    data_agenda,
    especialidade,
    horario,
    codigo,
    paciente,
    telefone,
    data_hora_enviar,
    data_hora_upload,
    data_hora_envio,
    customer_service_id,
    wa_message_id,
    resposta,
    dt_resposta,
    solicitante,
    nome_arquivo,
    id_usuario,
    nome_usuario,
    template_id,
    tipo_envio,
    exame
)
VALUES (
    $1, $2, $3, $4, $5, $6, $7, $8,
    NULL, NULL, NULL, NULL, NULL,
    $9, NULL,
    $10, NULL,
    $11, $12, $13, $14, $15
)
"""

In [128]:
await conn.executemany(insert_sql, registros)

print(f"{len(registros)} registros inseridos com sucesso!")

500 registros inseridos com sucesso!


In [129]:
await conn.close()